In [299]:
import numpy as np
from numpy.linalg import inv
import cmath
M=16;
N=4;
#x=[[70,0,30,151,4,80,34,15],[170,100,300,15,14,81,43,51],[70,101,301,15,14,8,56,79],[171,106,130,115,411,89,111,69],[70,110,30,151,4,8,34,15],[70,0,30,15,41,80,340,151],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0]]
#x=[[0,2,6,2,8,0,2,10],[8,0,8,4,2,2,8,4],[4,8,4,6,10,4,0,4],[0,0,8,2,0,2,8,6],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0]]
#QAM={0,4,8,12,16}
x=[[0,8,4,16],[0,0,12,16],[0,16,0,12],[4,0,8,16],[0,0,12,16],[0,4,12,8],[4,4,12,16],[8,4,16,12],[16,16,12,16],[4,4,8,8],[0,0,4,16],[0,0,8,16],[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]
#k_dash=[0,1,2];
k_dash=[0,3,4];
l_dash=[0,3,4];
#k_dash=[0,2,4];
#l_dash=[0,2,4];
#l_dash=[0,1,2];
#v_l_k=[[0.4,0.05],[1,0.3]];
v_l_k=[[4,5,8,5,1],[1,3,9,2,6],[0,12,5,2,7],[10,2,15,2,8],[0,2,15,2,17]];
l_max= max(l_dash)
M_d=M-l_max;
y=np.zeros((M,N), dtype=complex)
Y=np.zeros((M,N), dtype=complex)
z=cmath.exp(((2*cmath.pi)/(M*N))*1j);
def perodic_sinc(x,N):
    n=(N-1)/N;
    imj=cmath.pi*x*n;
    if x==0:
        s=cmath.sqrt(N);
    else:
        s= ((cmath.sin(cmath.pi*x))*cmath.exp((imj)*1j))/(cmath.sqrt(N)*(cmath.sin((cmath.pi*x)/N)));
    return s
def circulant( arr,  n):
       
    c = np.zeros((N,N), dtype=complex) 
    for k in range(n):
        c[k][0] = arr[k]
 
    for  i in range(1,n):
        for j in range(n):
            if (j - 1 >= 0):
                c[j][i] = c[j - 1][i - 1]
            else:
                c[j][i] = c[n - 1][i - 1]
             
         
 
    return c ;

def array(v,m,l,N):
    arr = [0 for element in range(N)] 
    for i in range (0,N):
        arr[i]= v[m][l][i];
    return arr

def array2(v,m,N):
    arr= [0 for element in range(N)]
    for i in range (0,N):
        arr[i]= v[m][i]
    return arr

v=np.zeros((M,l_max+1,N),dtype=complex)
for m in range(0,M):
    for l in range(0,len(l_dash)):
        for k in range (0,N):
             for k_i in range (0,len(k_dash)):
                 if m>=l_dash[l]:
                    v[m][l_dash[l]][k]= v[m][l_dash[l]][k]+np.round(v_l_k[l_dash[l]][k_dash[k_i]]*(z**(k_dash[k_i]*(m-l_dash[l])))*perodic_sinc(k_dash[k_i]-k,N),3);
                    
             v[m][l_dash[l]][k] = (v[m][l_dash[l]][k]/cmath.sqrt(N));      



In [300]:
print(y[1].shape)

(4,)


In [301]:
def K_m_l(v,m,l,N):
    K_m_l = np.zeros((N,N), dtype=complex) 
    K_m_l= np.matrix(circulant(array(v,m,l,N),N));                   
    return K_m_l

for m in range (0,M):
    va= np.zeros((N,1),dtype=complex)
    for l in range (0,len(l_dash)):
        if(m>=l_dash[l]):
            va= va+np.dot(np.matrix(K_m_l(v,m,l_dash[l],N)),np.matrix(array2(x,m-l_dash[l],N)).transpose())
            #print(m)
            #print(l)
            #print(m-l_dash[l])
    y[m]=(va.transpose())
print(y[1])
    


[  0.    +0.j     57.414+17.418j 135.64 +27.814j  78.784 +6.12j ]


In [302]:

for i in range (0,M):
    for j in range (0,N):
        Y[i][j]=np.round(y[i][j],2)
print(np.matrix(Y[1]))

[[  0.   +0.j    57.41+17.42j 135.64+27.81j  78.78 +6.12j]]


In [303]:
#############(MRC IN DELAY DOPPLER DOMAIN)###################
x_p= np.zeros((M,N), dtype=complex)
D_m=np.zeros((M,N,N), dtype=complex)
for m in range (0,M_d):
    for j in range (0,len(l_dash)):
            D_m[m]=np.add(D_m[m], np.dot((K_m_l(v,m+l_dash[j],l_dash[j],N)).getH(),K_m_l(v,m+l_dash[j],l_dash[j],N)))


i=1;

b= np.zeros((l_max+1,M,N), dtype=complex)
c= np.zeros((M,N), dtype=complex)
for i in range(0,i):
   for m in range (0,M_d):
       mtx= np.zeros((N,1), dtype=complex)
       ntx= np.zeros((N,1), dtype=complex)
       for l in range (0,len(l_dash)):
           w=np.zeros((N,1), dtype=complex)
           for p in range (0,len(l_dash)):
               if l_dash[p]!=l_dash[l]:
                   #a= (K_m_l(v,m+l_dash[l],l_dash[p],N)).transpose()
                   #x=np.matrix(x_p[m+l_dash[l]-l_dash[p]])
                   a= np.matrix(K_m_l(v,m+l_dash[l],l_dash[p],N))
                   x= np.matrix(x_p[m+l_dash[l]-l_dash[p]]).transpose()
                   #print(x.shape)
                   #print(a.shape)
                   w=np.add(w,(np.dot(a,x)))
                   #print((np.dot(a,x)).shape)
                 #print(np.matrix(x_p[m+l_dash[l]-l_dash[p]]).transpose())
                 #print(np.matrix(K_m_l(v,m+l_dash[l],l_dash[p],N)))
                 
           #print(w.shape)
           #print(np.matrix(Y[m+l_dash[l]]).transpose().shape) 
           b[l_dash[l]][m]=np.matrix(Y[m+l_dash[l]])- np.matrix(w).transpose();
          
           #print(l)
           #print(m)
           #print(np.matrix(Y[m+l_dash[l]]).shape)
           #print(w.transpose().shape)
          # print(np.matrix(b[l_dash[l]][m]).shape)
       for z in range (0,len(l_dash)):
               var1=(np.matrix(K_m_l(v,m+l_dash[z],l_dash[z],N)).getH())
               var2=np.matrix(b[l_dash[z]][m]).transpose()
           #print("------")
           #print(np.matrix(g[m]).shape)
           #print("------")
               mtx= np.add(mtx,(np.dot(var1,var2)))
               #print((np.dot(var1,var2)).shape)

       #print(mtx.shape)
           #print(var2.shape)
       
       #print("--------------------")
          # print(g[m].shape)
       ntx= np.dot(inv(np.matrix(D_m[m])),mtx)
       #print(np.dot(inv(np.matrix(D_m[m])),mtx).shape)
       x_p[m]=ntx.transpose()
for i in range (0,M):
    print (x_p[i])
    #print(x_p[i].shape)
                     

[ 0.11980796-0.20473627j  8.61236703+1.75803045j 12.9518944 +4.89022647j
 26.81172619+2.39373835j]
[-0.23793005+0.53887247j  8.19672361+1.24788582j 11.07211969+1.46420731j
 18.76293704-0.51438225j]
[ 2.36490211+0.19304517j 14.01043875+1.61673669j  7.22223814+0.85775459j
 19.20695376-1.9035378j ]
[ 2.93870599+0.77077283j  3.37446958+2.20192241j 10.73235007-1.77124489j
 18.0033534 -1.89353911j]
[ 2.15243501+2.35445626j  0.8847916 -1.41683598j 14.61878784-0.29967839j
 16.45153322+0.27168324j]
[ 1.64025904+0.66669704j  5.01681742-0.12687913j 13.19242625-1.4280788j
  9.18222633-2.18898442j]
[ 4.24257683-0.41364006j  5.09648035-0.77058222j 13.4672337 -1.12460504j
 12.15691259-1.35158754j]
[ 7.99852548-1.21536861j  4.27473761-0.70328765j 16.5384048 +1.64158377j
  8.94515687+0.30607141j]
[16.07499682-0.04768689j 14.59721236+0.24223128j 11.39870694+1.42663385j
 15.18302442+1.90971914j]
[4.30727966+0.26516652j 3.6391559 -0.40824195j 8.02227592+0.68670945j
 6.47918323+6.0080321j ]
[ 1.25060557+0.

In [ ]:
############( MP DETECTION ) ###############
